In [2]:
import keras
import csv
import scipy
import numpy as np
import nltk #from nltk.tokenize import punkt #.tokenize.moses import MosesTokenizer
from gensim.models import KeyedVectors
from itertools import islice
import pickle
import pandas as pd
import re
from nltk.corpus import stopwords
import json
import pickle
from cyordereddict import OrderedDict
#from joblib import Parallel, delayed
pattern = re.compile(r'[^\s\w_]+')

DATASET_FILE_PATH = "./DataSet/yelp.csv/yelp.csv"
WORD2VEC_FILE_PATH = "./DataSet/FBword2vec/wiki.en.vec"
TRAINING_DATA_PATH = "./DataSet/trainingData"
EXTENDED_DATASET_PATH = "./DataSet/review.json"

###File Names
TEST_NAME = "standard_tokenizer_No_Stopwords_noPunctuation_FB_vectors_100percentDataset"

EMBEDDED_MATRIX_NAME = TEST_NAME+'_embedded_matrix.pkl'
TOKENIZED_SENTENCES_NAME = TEST_NAME+ "_tokenized_sentences.pkl"
WORD_INDEX_NAME = TEST_NAME+ "_word_index.pkl"
INDEXED_TOKENIZED_SENTENCES_NAME = TEST_NAME + "_indexed_tokenized_sentences.pkl"

LABELS_NAME = 'labels.pkl'


AMOUNT_OF_WORDS = 80
FACEBOOK_EMBEDDING_SIZE = 300

C:\Users\skymont\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\skymont\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def load_word2vec(path):
    return KeyedVectors.load_word2vec_format(path)

In [3]:
 def vectorize(x,keyedvector):
        try: 
            return pd.Series(keyedvector.word_vec(x,use_norm=False).tolist())
        except KeyError:
            return []#[0]*FACEBOOK_EMBEDDING_SIZE

In [4]:
wordVector = load_word2vec(WORD2VEC_FILE_PATH)
wordVector.vocab

{',': <gensim.models.keyedvectors.Vocab at 0x1c2dc9bfe48>,
 '.': <gensim.models.keyedvectors.Vocab at 0x1c2d3447c50>,
 'the': <gensim.models.keyedvectors.Vocab at 0x1c2dc9bfe80>,
 '</s>': <gensim.models.keyedvectors.Vocab at 0x1c2dc9b0d30>,
 'of': <gensim.models.keyedvectors.Vocab at 0x1c2dc9bfef0>,
 '-': <gensim.models.keyedvectors.Vocab at 0x1c2dc9bf048>,
 'in': <gensim.models.keyedvectors.Vocab at 0x1c2dc9bff28>,
 'and': <gensim.models.keyedvectors.Vocab at 0x1c2dc9bf0f0>,
 "'": <gensim.models.keyedvectors.Vocab at 0x1c2dc9bff98>,
 ')': <gensim.models.keyedvectors.Vocab at 0x1c2dc9bf080>,
 '(': <gensim.models.keyedvectors.Vocab at 0x1c2dc9bffd0>,
 'to': <gensim.models.keyedvectors.Vocab at 0x1c2dc9bf198>,
 'a': <gensim.models.keyedvectors.Vocab at 0x1c390d0c0b8>,
 'is': <gensim.models.keyedvectors.Vocab at 0x1c2dc9bf160>,
 'was': <gensim.models.keyedvectors.Vocab at 0x1c390d0c128>,
 'on': <gensim.models.keyedvectors.Vocab at 0x1c2dc9bf208>,
 's': <gensim.models.keyedvectors.Vocab at

In [5]:
extended_dataset = pd.read_pickle("extended_dataframe.pkl")
#extended_dataset = extended_dataset.sample(frac=0.001)

In [6]:
def get_tokens(texts, embedding_dictionary_vocab):
    word_index = OrderedDict()
    tokenized_sentences = []
    stop_words = set(stopwords.words("english"))
    total_texts = len(texts)
    i=1
    for text in texts:
        if i%50000 == 0: print(str((i/total_texts)*100)+"% completed.")
        i+=1
        tokens = nltk.word_tokenize(pattern.sub("",text).lower())
        tokens = nltk.word_tokenize(text)
        tokens = [token for token in tokens if token not in stop_words and token in embedding_dictionary_vocab]
        word_index.update({token : None for token in tokens})
        tokenized_sentences.append(tokens)
        
    return tokenized_sentences, word_index.keys()

In [7]:
def make_embedding_matrix(keyedvector, word_index):
    i=0
    embedding_matrix = np.zeros((len(word_index), FACEBOOK_EMBEDDING_SIZE))
    for word in word_index:
        embedding_matrix[i] = keyedvector.word_vec(word,use_norm=False)
        i+=1
    return embedding_matrix

In [ ]:
embedded_matrix = make_embedding_matrix(wordVector, word_index)
embedded_matrix.shape

In [ ]:
np.save(EMBEDDED_MATRIX_NAME,embedded_matrix)

In [ ]:
with open(TOKENIZED_SENTENCES_NAME,'wb') as pickle_file:
    pickle.dump(tokenized_sentences,pickle_file)

In [ ]:
with open(WORD_INDEX_NAME,'wb') as pickle_file:
    pickle.dump(word_index,pickle_file)

In [5]:
def stringsToIndexValue(tokenized_sentences, word_index):
    indexer={word: index for (word, index) in zip(word_index,range(len(word_index)))}
    print("Done with dictionary")
    #indexed_tokenized_sentences = Parallel(n_jobs=4)(delayed(lambda x: [indexer[word] for word in x])(sentence) for sentence in tokenized_sentences)
    return [[indexer[word] for word in sentence] for sentence in tokenized_sentences]
    

In [3]:
tokenized_sentences = pickle.load(open(TOKENIZED_SENTENCES_NAME,'rb'))
print('sentences_loaded')
embedded_matrix =  np.load(EMBEDDED_MATRIX_NAME+'.npy')
print('embedded matrix loaded')
word_index = pickle.load(open(WORD_INDEX_NAME,'rb'))
print('word index loaded')

sentences_loaded
embedded matrix loaded
word index loaded


In [6]:
indexed_tokenized_sentences = stringsToIndexValue(tokenized_sentences, word_index)
len(indexed_tokenized_sentences)

Done with dictionary


5261669

In [7]:
with open(INDEXED_TOKENIZED_SENTENCES_NAME,'wb') as pickle_file:
    pickle.dump(indexed_tokenized_sentences,pickle_file)